In [34]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
import numpy as np
import cv2

In [35]:
import tflearn
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

## Loading the model

Models with Keras

In [36]:
def select_model(model):
    models = ['keras_model.h5', 'model_upb.h5', 'model_upb_2.h5', 'model_upb_3.h5']
    return load_model(models[model - 1], compile=False) 

In [37]:
def config_img(model, img):
    if model == 1:
        img = cv2.resize(img, (224, 224))
        img = img.reshape(1, 224, 224, 3)
    else:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (50, 50))
        img = img.reshape(1, 50, 50, 1)
    return img

In [38]:
facedetect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
font = cv2.FONT_HERSHEY_COMPLEX

model_chosen = 4

model = select_model(model_chosen)

class_names = {
     0: 'ALEJANDRA',
     1: 'MAURICIO',
     2: 'SEBASTIAN',
     3: 'IGNACIO'
}

while True:
    success, img_original = cap.read()
    faces = facedetect.detectMultiScale(img_original, 1.3, 5)
    for x, y, w, h in faces:
        crop_img = img_original[y:y+h, x:x+w]
        img = config_img(model_chosen, crop_img)
        prediction = model.predict(img)
        class_index = np.argmax(prediction)
        probability = np.amax(prediction)

        cv2.rectangle(img_original, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(img_original, class_names[class_index], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        cv2.putText(img_original, str(int(probability*100))+"%", (x+w+20, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    try:
        cv2.imshow('Modelo '+str(model_chosen), img_original)
    except:
        pass

    key = cv2.waitKey(1) & 0xFF

    if key == ord('q'):
        break
    elif key in [ord('1'), ord('2'), ord('3'), ord('4')]:
        model_chosen = int(chr(key))
        model = select_model(model_chosen)

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

2023-11-06 01:05:22.821630: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_34_2/bias/Assign' id:57367 op device:{requested: '', assigned: ''} def:{{{node dense_34_2/bias/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_34_2/bias, dense_34_2/bias/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
2023-11-06 01:05:25.801536: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_35_2/Softmax' id:57410 op device:{requested: '', assigned: ''} def:{{{node dense_35_2/Softmax}} = Softmax[T=DT_FLOAT, _has_manual_control_dependencies=true](dense_35_2/BiasAdd)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or creat

-1